In [19]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp('I love learning Python and spaCy!')

print(f"{'Text':<10} {'POS_':<12} {'Tag_':<8} {'Dep_':<14} {'Lemma_':<10}")
print("-" * 60)

for token in doc:
    # 打印每个token的文本、词性、详细标签、依存关系和词根原型
    print(f"{token.text:<10} {token.pos_:<12} {token.tag_:<8} {token.dep_:<14} {token.lemma_:<10}")


Text       POS_         Tag_     Dep_           Lemma_    
------------------------------------------------------------
I          PRON         PRP      nsubj          I         
love       VERB         VBP      ROOT           love      
learning   VERB         VBG      xcomp          learn     
Python     PROPN        NNP      dobj           Python    
and        CCONJ        CC       cc             and       
spaCy      NOUN         NNS      conj           spaCy     
!          PUNCT        .        punct          !         


In [18]:
    
GENERIC_STOP_WORDS = {
    "experience", "skill", "ability", "knowledge", "understanding",
    "responsibility", "role", "work", "developer", "engineer", "candidate",
    "team", "year", "we", "you", "the", "a", "an", "and", "or", "in", "on", "at"
}
TECH_WHITELIST = {"java", "go"}
    
def extract_tech_nouns(jd_text):
  jdDoc = nlp(jd_text)
  jdSet = set()
  for jdToken in jdDoc:
    if jdToken.pos_ not in jdToken.pos_ not in ['DET', 'ADP']:
      continue
    if jdToken.pos_ in ['NOUN', 'PROPN'] and jdToken.text.lower() not in GENERIC_STOP_WORDS and len(jdToken.text) > 2:
      jdSet.add(jdToken.text)
    elif jdToken.text.lower() in TECH_WHITELIST:
      jdSet.add(jdToken.text)
  return list(jdSet)

extract_tech_nouns("We need a Python developer with experience in Docker, Kubernetes, and Go. Java experience is a plus.")

['Docker', 'Kubernetes', 'Java', 'plus', 'Go', 'Python']

In [21]:
import spacy

# 加载官方中文模型
nlp = spacy.load("zh_core_web_lg")  # 根据你下载的模型调整名称

# 测试分词
test_text = "我喜欢用Python编程和Docker部署项目。"
doc = nlp(test_text)
print("分词结果：", [token.text for token in doc])
# 预期输出：['我', '喜欢', '用', 'Python', '编程', '和', 'Docker', '部署', '项目', '。']

分词结果： ['我', '喜欢', '用', 'Python', '编程', '和', 'Docker', '部署', '项目', '。']


In [ ]:
import re
import spacy

# 加载官方中文模型
nlp = spacy.load("zh_core_web_lg")  # 根据你下载的模型调整名称


def protect_english_terms(text):
    """
    保护英文术语，防止被错误分词
    将连续的大写字母、数字、点号组成的术语用特殊标记保护起来
    """
    # 匹配可能的技术术语：字母、数字、点、减号组成的连续字符串
    pattern = r'\b[A-Za-z][A-Za-z0-9.-]{1,}\b'
    protected_terms = {}
    
    def replace_match(match):
        term = match.group()
        placeholder = f"__TERM_{len(protected_terms)}__"
        protected_terms[placeholder] = term
        return placeholder
    
    # 先用占位符替换所有英文术语
    protected_text = re.sub(pattern, replace_match, text)
    return protected_text, protected_terms

def restore_terms(text, protected_terms):
    """将保护起来的术语恢复回来"""
    for placeholder, term in protected_terms.items():
        text = text.replace(placeholder, term)
    return text

# 测试保护功能
test_text = "需要熟悉Redis、MySQL和K8s技术，有AWS经验。"
protected_text, term_dict = protect_english_terms(test_text)
print("保护后:", protected_text)
print("术语映射:", term_dict)

# 使用更精确的模式库
CHINESE_TECH_PATTERNS = {
    '云服务': ['阿里云', '腾讯云', '华为云', 'AWS', 'Azure', '云原生', '云计算'],
    '数据库': ['MySQL', 'Redis', 'MongoDB', 'PostgreSQL', 'Oracle', '数据库'],
    '中间件': ['Kafka', 'RabbitMQ', 'RocketMQ', '中间件', '消息队列'],
    '容器': ['Docker', 'Kubernetes', 'K8s', '容器', '容器化'],
    '框架': ['Spring', 'Django', 'Flask', 'React', 'Vue', '微服务', '分布式']
}


def contains_tech_pattern(word, patterns_dict):
    """更严格的中文技术模式匹配"""
    for category, patterns in patterns_dict.items():
        for pattern in patterns:
            # 只有当词语完全匹配或包含重要技术模式时才认为是技术词
            if word == pattern or (len(pattern) > 1 and pattern in word):
                return True, category
    return False, None

# 测试
test_words = ['阿里云', '数据库设计', '消息中间件', '普通经验']
for word in test_words:
    is_tech, category = contains_tech_pattern(word, CHINESE_TECH_PATTERNS)
    print(f"'{word}': 技术词={is_tech}, 类别={category}")
    


def enhanced_chinese_keyword_extraction(jd_text):
    """
    增强版中文技术关键词提取
    """
    # 1. 预处理：保护英文术语
    protected_text, protected_terms = protect_english_terms(jd_text)
    
    # 2. spaCy处理
    doc = nlp(protected_text)
    
    # 3. 恢复术语并提取
    keywords = set()
    stop_words = {'经验', '要求', '熟悉', '优先', '以上', '团队', '沟通', '负责'}
    
    for token in doc:
        # 恢复原始术语
        original_text = token.text
        if token.text.startswith('__TERM_'):
            original_text = protected_terms.get(token.text, token.text)
        
        # 跳过停用词和标点
        if original_text in stop_words or token.pos_ == 'PUNCT':
            continue
            
        # 策略1：实体识别
        if token.ent_type_ in ['ORG', 'PRODUCT', 'GPE']:
            keywords.add(original_text)
            
        # 策略2：英文术语（包括恢复后的）
        elif original_text.isascii() and len(original_text) > 1:
            keywords.add(original_text)
            
        # 策略3：中文技术模式匹配（你的建议！）
        elif not original_text.isascii():  # 如果是中文
            is_tech, category = contains_tech_pattern(original_text, CHINESE_TECH_PATTERNS)
            if is_tech:
                keywords.add(original_text)
                
        # 策略4：特定技术名词
        elif token.pos_ == 'NOUN' and original_text in ['容器', '微服务', '分布式', '缓存']:
            keywords.add(original_text)
    
    # 4. 后处理：确保常见技术词被包含
    tech_whitelist = ['redis', 'mysql', 'python', 'java', 'docker', 'kubernetes', 'k8s', 'aws']
    for term in tech_whitelist:
        if term in jd_text.lower():
            keywords.add(term)
    
    return list(keywords)

# 测试
test_jd = """
需要熟悉Redis、MySQL和K8s技术，有AWS经验。
熟悉阿里云平台操作，了解数据库优化。
具备微服务架构设计能力。
"""


result = enhanced_chinese_keyword_extraction(test_jd)
print("增强版提取结果:", result)
# 预期应该包含: ['Redis', 'MySQL', 'K8s', 'AWS', '阿里云', '数据库', '微服务']

保护后: 需要熟悉Redis、MySQL和K8s技术，有AWS经验。
术语映射: {}
'阿里云': 技术词=True, 类别=云服务
'数据库设计': 技术词=True, 类别=数据库
'消息中间件': 技术词=True, 类别=中间件
'普通经验': 技术词=False, 类别=None
增强版提取结果: ['阿里云', 'mysql', 'is', 'k8s', 'AWS', 'Red', 'aws', '数据库', 'redis', 'MySQL', 'K8s']


In [48]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import os

load_dotenv()

client = OpenAI(
    api_key = os.getenv('OPENAI_API_KEY'),
    base_url = os.getenv('OPENAI_BASE_URL'),
)
model_name = os.getenv('MODEL_NAME')

def parse_jd_with_llm(jd_text):
    """
    使用LLM解析JD，返回结构化数据
    """
    prompt = f"""
你是一个资深的HR专家和技术招聘者。你的任务是从下面的职位描述（JD）中精确提取信息，并严格按照指定的JSON格式输出。

# 提取要求：
1.  job_title: 提取职位名称（如：前端开发工程师、后端开发工程师）。
2.  skills: 提取所有提到的技术技能、编程语言、工具和框架。这是一个数组。
3.  experience_years: 提取工作经验要求，通常是一个数字或数字范围（如"3年"提取为3，"3-5年"提取为"3-5"）。如果没有明确要求，则为null。
4.  education: 提取学历要求（如"本科及以上"）。如果没有明确要求，则为null。
5.  responsibilities: 提取职位核心职责，总结为2-5个要点。这是一个数组。
6.  nice_to_have: 提取"优先考虑"、"加分项"、"有以下经验者优先"后面的内容。这是一个数组。

# 输出格式：
你必须严格输出一个且仅一个JSON对象，不要有任何其他解释。

# 职位描述：
{jd_text}
"""

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0  # 温度设为0，确保输出确定性
        )
        
        # 从AI的回复中提取JSON部分
        result_text = response.choices[0].message.content
        # 解析JSON字符串为Python字典
        structured_data = json.loads(result_text)
        return structured_data
        
    except json.JSONDecodeError as e:
        print("解析JSON失败，AI返回了非JSON内容：")
        print(result_text)
        return None

# 测试
jd_text = """
职位名称：高级前端开发工程师

职位描述：
我们正在寻找一位具有3年以上前端开发经验的高手。你需要负责核心产品的前端架构设计与开发，主要使用React和TypeScript技术栈。

职位要求：
- 精通JavaScript/TypeScript，熟练掌握React及其生态圈
- 有3-5年大型前端项目开发经验
- 本科及以上学历，计算机相关专业优先
- 有性能优化经验者优先
- 熟悉Webpack、Vite等构建工具
- 有小程序开发经验者优先

职位职责：
- 负责公司主要产品的前端模块开发
- 参与技术选型和架构设计
- 优化前端性能，提升用户体验
- 与后端工程师协作完成接口联调
"""

result = parse_jd_with_llm(jd_text)
if result:
    print(json.dumps(result, ensure_ascii=False, indent=2))

NotFoundError: Error code: 404 - {'error': {'message': 'Not found the model gpt-3.5-turbo or Permission denied', 'type': 'resource_not_found_error'}}